In [41]:
import requests
import pandas as pd
import numpy as np
import io
from datetime import datetime, timedelta
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

# Constants
API_KEY = '2lZRFGaqFiEYkzr7WUuT4EaoC1X'
SINCE_DATE = int(datetime(2017, 1, 1).timestamp())
UNTIL_DATE = int(datetime.now().timestamp())

# URLs for fetching data
METRICS = [
    'https://api.glassnode.com/v1/metrics/market/mvrv'
]

# List of all assets (split into chunks for readability)
ASSETS = [
    'BTC', 'ETH', 'LTC', '1INCH', 'AAVE', 'ABT', 'ACH', 'ACX'
]

def fetch_glassnode_data(url, asset='BTC'):
    """
    Fetches data from Glassnode API for a specific metric and asset.
    """
    params = {
        'a': asset,
        's': SINCE_DATE,
        'u': UNTIL_DATE,
        'api_key': API_KEY,
        'f': 'CSV',
        'c': 'USD'
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        df = pd.read_csv(io.StringIO(response.text))
        metric_name = url.split('/')[-1]
        column_name = f"{metric_name}_{asset}"
        df.columns = ['t', column_name]
        df['t'] = pd.to_datetime(df['t'], unit='s')
        df[column_name] = pd.to_numeric(df[column_name], errors='coerce')
        return df
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data for {asset} from {url}: {e}")
        return None
    except Exception as e:
        print(f"Unexpected error processing data for {asset} from {url}: {e}")
        return None

In [42]:

def fetch_all_data():
    """
    Fetches data for all metrics and assets and combines them into a single DataFrame.
    """
    all_dfs = []
    for metric_url in METRICS:
        for asset in ASSETS:
            print(f"Fetching {metric_url.split('/')[-1]} for {asset}")
            df = fetch_glassnode_data(metric_url, asset)
            if df is not None:
                all_dfs.append(df)

    if not all_dfs:
        raise Exception("No data was successfully fetched")

    merged_df = all_dfs[0]
    for df in all_dfs[1:]:
        merged_df = pd.merge(merged_df, df, on='t', how='outer')

    merged_df.set_index('t', inplace=True)
    return merged_df

try:
    final_df = fetch_all_data()
    print("Data collection complete")
    print(f"DataFrame shape: {final_df.shape}")
    print("\nColumns in the dataset:")
    print(final_df.columns.tolist())
except Exception as e:
    print(f"Error in data collection process: {e}")

Fetching mvrv for BTC
Fetching mvrv for ETH
Fetching mvrv for LTC
Fetching mvrv for 1INCH
Fetching mvrv for AAVE
Fetching mvrv for ABT
Fetching mvrv for ACH
Fetching mvrv for ACX
Data collection complete
DataFrame shape: (2863, 8)

Columns in the dataset:
['mvrv_BTC', 'mvrv_ETH', 'mvrv_LTC', 'mvrv_1INCH', 'mvrv_AAVE', 'mvrv_ABT', 'mvrv_ACH', 'mvrv_ACX']


In [43]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [44]:
def plot_all_mvrv(df, initial_scale='linear', smoothing=30):
    """
    Create an interactive Plotly figure showing all MVRV ratios
    
    Parameters:
    df (pandas.DataFrame): DataFrame containing MVRV columns
    initial_scale (str): Initial y-axis scale ('linear' or 'log')
    smoothing (int): Window size for Simple Moving Average smoothing. Set to 0 to disable.
    """
    # Get all MVRV columns
    mvrv_columns = [col for col in df.columns if col.startswith('mvrv_')]
    
    # Create a copy of the dataframe for smoothing
    if smoothing > 0:
        df_smooth = df.copy()
        for col in mvrv_columns:
            df_smooth[col] = df[col].rolling(window=smoothing, min_periods=1).mean()
    else:
        df_smooth = df
    
    # Create the figure
    fig = go.Figure()
    
    # Generate colors using plotly's built-in color sequences
    import plotly.express as px
    colors = px.colors.qualitative.Set3
    color_idx = 0
    num_colors = len(colors)
    
    # Add watermark using annotation
    fig.add_annotation(
        text="_cryptovizart",
        xref="paper",
        yref="paper",
        x=0.5,
        y=0.5,
        showarrow=False,
        font=dict(size=100, color="rgba(200,200,200,0.2)"),
        textangle=0
    )
    
    # Add traces for each MVRV
    for column in mvrv_columns:
        crypto_name = column.replace('mvrv_', '')
        
        # Set color and line style for the trace
        if crypto_name == 'BTC':
            color = '#000000'  # Dark black for Bitcoin
            line_style = 'dash'  # Dashed line for Bitcoin
        else:
            color = colors[color_idx % num_colors]
            line_style = 'solid'  # Solid line for others
            color_idx += 1
        
        fig.add_trace(
            go.Scatter(
                x=df_smooth.index,
                y=df_smooth[column],
                name=crypto_name,
                line=dict(
                    color=color,
                    dash=line_style,
                    width=2  # Make lines more visible
                ),
                mode='lines',
                connectgaps=False  # Don't connect gaps in the data
            )
        )
    
    # Add y=1 reference line
    fig.add_hline(
        y=1,
        line_dash="dash",
        line_color="gray",
        opacity=0.7,
        name="MVRV = 1"
    )
    
    # Update layout
    fig.update_layout(
        title=f'Cryptocurrency MVRV Ratios Over Time (SMA-{smoothing})' if smoothing > 0 
              else 'Cryptocurrency MVRV Ratios Over Time',
        xaxis_title='Date',
        yaxis_title='MVRV Ratio',
        template='none',  # Remove template to start with clean slate
        plot_bgcolor='white',  # Set plot background to white
        paper_bgcolor='white',  # Set paper background to white
        hovermode='x unified',
        showlegend=True,
        legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=1.05
        ),
        # Make the figure responsive
        height=800,
        width=1200,
        margin=dict(r=150),  # Add right margin for legend
        # Set initial y-axis scale
        yaxis_type=initial_scale,
        # Remove gridlines
        xaxis=dict(
            showgrid=False,
            zeroline=False,
            showline=True,
            linewidth=1,
            linecolor='black',
            mirror=True
        ),
        yaxis=dict(
            showgrid=False,
            zeroline=False,
            showline=True,
            linewidth=1,
            linecolor='black',
            mirror=True
        )
    )
    
    # Add range slider
    fig.update_xaxes(rangeslider_visible=True)
    
    return fig

In [45]:

# Usage example:
combined_fig = plot_all_mvrv(final_df)
# Display the figures
combined_fig.show()
